In [ ]:
import requests
import pandas as pd

# 1. Configuration
API_KEY = 'aad4cbdae56d6d693c4f99064fe46dcd'
CITY = 'Ibadan'  # Choose a city relevant to your economic study
URL = f'http://api.openweathermap.org/data/2.5/forecast?q={}&appid={API_KEY}&units=metric'

# 2. Fetch Data
response = requests.get(URL)
data = response.json()

if response.status_code == 200:
    print(f"Successfully fetched weather data for {CITY}!")
else:
    print(f"Error: {data.get('message', 'Could not connect')}")

In [ ]:
# 3. Extract the "List" of weather readings
weather_list = data['list']

# 4. Flatten the data into a Pandas Table
# We extract Date, Temperature, Humidity, and Pressure
df = pd.json_normalize(weather_list)

# 5. Clean up the Column Names for easier analysis
df = df[['dt_txt', 'main.temp', 'main.humidity', 'main.pressure', 'weather']]
df.columns = ['Date', 'Temp_C', 'Humidity', 'Pressure', 'Description']

print("Preview of your Economic Dataset:")
df.head()

In [ ]:
volatility = df['Temp_C'].std()

In [ ]:
print(f"Temperature Volatility: {volatility}")

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
weekly_avg = df.resample('W', on='Date').mean()

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')

In [ ]:
import matplotlib.pyplot as plt

# Plotting Temperature over time
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Temp_C'],
         label='Daily Temperature', color='blue', marker='o')
plt.title('Weather Trend Analysis for Economic Forecasting', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.legend()
plt.show()